# 首先，請先安裝必要的套件：

In [ ]:
!pip install langgraph openai

# 構建我們的 LangGraph 工作流程。

In [ ]:
# 1. 導入必要的模組與套件
from langgraph.graph import StateGraph, END
from dataclasses import dataclass
from typing import Dict
from openai import OpenAI
import os

In [ ]:
# 2. 定義「狀態（State）」結構：AI的記憶體
@dataclass
class ReflectionState:
    draft: str # 原始草稿
    email: str = "" # 正在處理的郵件內容
    feedback: str = "" # 反思節點給出的改進建議
    has_issue: bool = False # 判斷是否有問題需要修正

In [ ]:
# 3. 初始化 OpenAI 客戶端
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [ ]:
# 4. 定義「節點（Nodes）」：AI執行的任務
def llm_call(prompt: str, max_tokens: int = 200) -> str:
    # 這是實際呼叫 OpenAI 模型的函式
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens
    )
    return response.choices[0].message.content.strip()

def generate_node(state: ReflectionState) -> Dict:
    """生成節點：改寫郵件草稿為專業語氣"""
    prompt = f"將以下電子郵件草稿改為專業語氣：\n{state.draft}"
    email_draft = llm_call(prompt)
    
    # 更新狀態：將初稿存入 email
    state.email = email_draft
    return {"email": state.email}

def reflect_node(state: ReflectionState) -> Dict:
    """反思節點：檢查語氣與結構，並給予建議"""
    prompt = f"檢查以下電子郵件是否存在語氣或結構問題：\n{state.email}\n若有問題，回覆具體建議；若無，回覆'無問題'"
    feedback_text = llm_call(prompt)
    
    # 判斷是否有問題
    has_issue = "無問題" not in feedback_text
    
    # 更新狀態
    state.feedback = feedback_text
    state.has_issue = has_issue
    return {"feedback": state.feedback, "has_issue": state.has_issue}

def update_node(state: ReflectionState) -> Dict:
    """更新節點：根據反饋修正郵件"""
    prompt = f"基於以下建議修正電子郵件：\n建議：{state.feedback}\n原始郵件：\n{state.email}"
    updated_email = llm_call(prompt)
    
    # 更新狀態
    state.email = updated_email
    return {"email": state.email}

In [ ]:
# 5. 構建「圖（Graph）」與「邊（Edges）」：這是AI的工作流程
workflow = StateGraph(ReflectionState)

# 新增節點到工作流程中
workflow.add_node("generate", generate_node)
workflow.add_node("reflect", reflect_node)
workflow.add_node("update", update_node)

# 設定進入點
workflow.set_entry_point("generate")

# 無條件邊：從 "generate" 到 "reflect"
workflow.add_edge("generate", "reflect")

# 條件邊：從 "reflect" 節點根據是否有問題來決定下一步
workflow.add_conditional_edges(
    "reflect",
    # 判斷函數：如果有問題，進入 "update"，否則結束
    lambda state: "update" if state.has_issue else "end",
    {"update": "update", "end": END}
)

# 無條件邊：從 "update" 到 "reflect"，進行新一輪的檢查
workflow.add_edge("update", "reflect")

# 編譯工作流程，準備執行
graph = workflow.compile()


# 執行範例與成果展示

In [ ]:
# 執行範例
result = graph.invoke(ReflectionState(draft="嗨，明天可以開會嗎？"))

print(f"最終郵件：\n{result['email']}")
print(f"\n反饋紀錄：{result['feedback']}")
